# Data Cleaning and Processing

#### This notebook is designed to handle data processing tasks and extract information required for analysis. The steps involved are as follows:

- Feature Selection: Identifying and selecting the relevant features from the dataset.
- Imputing Missing Values: Addressing missing data by replacing them using appropriate methods.
- Removing Possible Duplicates: Ensuring data integrity by eliminating any potential duplicate entries, especially if not adequately handled during the data collection phase.
- Normalizing and Encoding Categorical Variables: Standardizing categorical variables and encoding them for further analysis.
- Exporting Cleaned Data: The processed data will then be exported in a suitable format for subsequent exploratory data analysis (EDA).

#### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

#very important
#Ignore warnings
from warnings import filterwarnings
filterwarnings('ignore')

#### Loading data

In [2]:
# pd.read_csv read data from the csv file and creates the data frame
master_df = pd.read_csv('../EDA/data/master_df.csv')

In [3]:
#checking the dimension of the dataframe
master_df.shape

(19835, 10)

#### Understanding the structure and properties of the data

In [4]:
#checking columns
# .columns returns the column names in the data frame
master_df.columns

Index(['id', 'subreddit', 'title', 'posttext', 'time_posted', 'time_now',
       'time_difference', 'num_comments', 'upvote_ratio', 'url'],
      dtype='object')

In [5]:
master_df.rename(columns={'posttext': 'selftext'}, inplace=True)

In [6]:
#checking dtypes
# .dtypes returns data types of each column in the data frame
master_df.dtypes

id                  object
subreddit           object
title               object
selftext            object
time_posted         object
time_now            object
time_difference     object
num_comments       float64
upvote_ratio       float64
url                 object
dtype: object

In [7]:
# .info : provides with the overview of the dataset
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19835 entries, 0 to 19834
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               19835 non-null  object 
 1   subreddit        19835 non-null  object 
 2   title            19835 non-null  object 
 3   selftext         19676 non-null  object 
 4   time_posted      19835 non-null  object 
 5   time_now         19835 non-null  object 
 6   time_difference  19835 non-null  object 
 7   num_comments     15382 non-null  float64
 8   upvote_ratio     15382 non-null  float64
 9   url              15382 non-null  object 
dtypes: float64(2), object(8)
memory usage: 1.5+ MB


In [8]:
master_df.head(2)

,id,subreddit,title,selftext,time_posted,time_now,time_difference,num_comments,upvote_ratio,url
0,18888ww,stocks,Rate My Portfolio - r/Stocks Quarterly Thread ...,Please use this thread to discuss your portfol...,2023-12-01 10:00:24,2024-02-28 22:08:57.699800,89 days 12:08:33.699801,621.0,0.99,https://www.reddit.com/r/stocks/comments/18888...
1,1b22xgu,stocks,"r/Stocks Daily Discussion Wednesday - Feb 28, ...",These daily discussions run from Monday to Fri...,2024-02-28 10:30:09,2024-02-28 22:08:57.699812,0 days 11:38:48.699813,387.0,0.88,https://www.reddit.com/r/stocks/comments/1b22x...


### Imputing Missing Values

#### Posttext (actual name was selftext)

In [9]:
master_df['selftext'].fillna('title', inplace = True)

In [10]:
master_df['selftext'].value_counts()
print(f'Self Text Null is: {master_df["selftext"].isnull().sum()}')

Self Text Null is: 0


In [11]:
master_df.isnull().sum()

id                    0
subreddit             0
title                 0
selftext              0
time_posted           0
time_now              0
time_difference       0
num_comments       4453
upvote_ratio       4453
url                4453
dtype: int64

In [12]:
#filling url missing values as 'unknown' 
master_df['url'] = master_df['url'].fillna('unknown')

In [13]:
master_df.isnull().sum()

id                    0
subreddit             0
title                 0
selftext              0
time_posted           0
time_now              0
time_difference       0
num_comments       4453
upvote_ratio       4453
url                   0
dtype: int64

In [14]:
master_df['num_comments'] = master_df['num_comments'].fillna(0)

In [15]:
master_df['upvote_ratio'] = master_df['upvote_ratio'].fillna(0)

In [16]:
master_df.isnull().sum()

id                 0
subreddit          0
title              0
selftext           0
time_posted        0
time_now           0
time_difference    0
num_comments       0
upvote_ratio       0
url                0
dtype: int64

## Removing duplicate values

In [17]:
#master_df.duplicated(title).sum()
master_df.duplicated(subset=['title']).sum()

15943

In [18]:
master_df.drop_duplicates(subset=['selftext'], inplace=True)

master_df.duplicated(subset=['selftext']).sum()

0

In [19]:
master_df.isna().sum()

id                 0
subreddit          0
title              0
selftext           0
time_posted        0
time_now           0
time_difference    0
num_comments       0
upvote_ratio       0
url                0
dtype: int64

In [20]:
master_df.shape

(3802, 10)

In [21]:
def preprocess_dataframe(df):
    # Normalize numerical columns
    numerical_cols = ['upvote_ratio', 'num_comments']
    df[numerical_cols] = StandardScaler().fit_transform(df[numerical_cols])

    # Process 'time_posted' column (timestamp)
    df['time_posted'] = pd.to_datetime(df['time_posted'])  # No need to specify unit

    # Extract hour_of_day and day_of_week from 'time_posted'
    df['hour_of_day'] = df['time_posted'].dt.hour
    df['day_of_week'] = df['time_posted'].dt.dayofweek

    # Drop 'time_posted' column after extracting hour_of_day and day_of_week
    df = df.drop('time_posted', axis=1)

    return df

In [22]:
#Checking the cleaned data
processed_df = preprocess_dataframe(master_df)
print(processed_df.shape)
print(processed_df.tail(10))

(3802, 11)
            id  subreddit                                              title  \
17624  1b1ntmx  investing                          What to do with lump sum?   
17625  1b1nakn  investing                       Long Term investment advice?   
17626  1b1mpq5  investing  Stock Market Hasn’t Been This Hot in Decades. ...   
17627  1b1mco5  investing  Broker for my child's UTMA hasn't made anythin...   
17628  1b1mcn0  investing  Why is it said that saving in the bank is the ...   
17629  1b1lcxt  investing  Warren Buffett is not fond of how the markets ...   
17630  1b1lb90  investing                     Schwab ? Making the change 😬 ?   
17631  1b1l921  investing              Short term (1month) cash, what to do?   
17662  1b0t1pn  investing  EV stocks. How are you navigating the current ...   
18504  1b2aue5  investing                 Hedging S&P 500 Position - 3 Years   

                                                selftext  \
17624  Curious what the general suggestion of th

In [23]:
## Saving the ouput processed data for further analysis

In [24]:
#saving the output as csv, named 'processed_data
processed_df.to_csv('../ANALYSIS/data/processed_data.csv', index=False)